In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinLibertine_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.7)
plt.rcParams["mathtext.fontset"] = "stix"

In [ ]:
figs = {}
data = pd.concat([pd.read_csv(data, names=[
    "Mode", "Variant", "Number of Processes", "Latency"
]) for data in glob('saved/mutex/*.txt')], ignore_index=True)
def official_variant(row):
    return {"Untrusted": "Unsafe", "Replicated": "PBFT", "Quorum": "COVER$_{Q}$", "NitroEnclaves": "COVER$_{NE}$"}[row.at["Variant"]]
data["Variant2"] = data.apply(official_variant, axis=1)
# data

# Latency distribution of every processor requesting once

Notice the significant larger spans of replicated. All requesting are concurrent while there is a sequential causality chain of processor#1 requested-processor#1 releasing-processor#2 requested-processor#2 releasing-processor#3 requested-... since the first processor is granted with the resource. The large spans are clear indicator of replicated being a more serial system.

In [ ]:
# plt.figure(figsize=(12, 3))
f = sns.pointplot(data=data[data["Mode"] == "All"],
    x="Number of Processes", y="Latency", hue="Variant2", 
    errorbar="sd",
    hue_order=["Unsafe", "PBFT", "COVER$_{Q}$", "COVER$_{NE}$"])
f.get_legend().remove()
# sns.move_legend(f, "upper right")
f.set(ylabel="Latency (s)", ylim=(0, 19.9))
f.set(xticks=[2 * x + 1 for x in range(8)])
figs["mutex-all"] = f

# Latency of only one processor requesting

In [ ]:
f = sns.lineplot(data=data[data["Mode"] == "One"],
    x="Number of Processes", y="Latency", hue="Variant2", 
    hue_order=["Unsafe", "PBFT", "COVER$_{Q}$", "COVER$_{NE}$"],
    style="Variant2", dashes=False, markers=["o"])
f.get_legend().set_title(None)
sns.move_legend(f, "lower center", ncol=4, bbox_to_anchor=(.5, 1), columnspacing=0.1, frameon=False)
f.set(ylim=(0, 1.79))
f.set(ylabel="Latency (s)")
figs["mutex-one"] = f

In [ ]:
for name, f in figs.items():
    f.figure.savefig(f"{name}.pdf", bbox_inches='tight')